# Example 1: Formula Creation

In [1]:
from mlisplacement.synthdoc.structs.airports import CHARGES as AIRPORTS_CHARGES
from mlisplacement.synthdoc.structs.airports import VARIABLES as AIRPORTS_VARIABLES
from mlisplacement.synthdoc.structs.azure import CHARGES as AZURE_CHARGES
from mlisplacement.synthdoc.structs.azure import VARIABLES as AZURE_VARIABLES

from mlisplacement.synthdoc.function_creation import create_charge_func, generate_output_structure

In [2]:
AZURE_VARIABLES

{'cpu_usage': {'description': 'The CPU usage during processing.',
  'dtype': float,
  'values': (0, 100.0),
  'units': 'percentage',
  'alternative_names': ['CPU load', 'CPU utilization']},
 'processing_time': {'description': 'The time taken for processing.',
  'dtype': float,
  'values': (0, 3600.0),
  'units': 'seconds',
  'alternative_names': ['processing duration', 'execution time']},
 'cpu_count': {'description': 'The number of CPUs used for processing.',
  'dtype': int,
  'values': (0, 64),
  'units': 'CPUs',
  'alternative_names': ['CPU cores', 'processor count']},
 'cpu_type': {'description': 'The type of CPU used for processing.',
  'dtype': str,
  'values': ['Intel', 'AMD', 'ARM'],
  'units': 'CPU type',
  'alternative_names': ['processor type', 'CPU model']}}

In [3]:
AZURE_CHARGES

{'cpu_compute': {'description': 'The charge associated with a CPU compute instance.',
  'variables': ['cpu_usage', 'processing_time', 'cpu_count', 'cpu_type'],
  'synonyms': ['CPU time', 'processing hours']}}

The following function creates a charge-function for a chosen charge

In [4]:
create_charge_func("cpu_compute", AZURE_CHARGES, AZURE_VARIABLES)

('cpu_compute conditions by cpu_type and cpu_count:\n\nfor Intel the following charges apply:\n  - if cpu_count in [0, 39), charge is 0 x cpu_count\n  - if cpu_count in [39, 57), charge is 1 x cpu_count\n  - if cpu_count in [57, 64), charge is 2 x cpu_count\nfor AMD the following charges apply:\n  - if cpu_count in [0, 39), charge is 0 x cpu_count\n  - if cpu_count in [39, 57), charge is 2 x cpu_count\n  - if cpu_count in [57, 64), charge is 4 x cpu_count\nfor ARM the following charges apply:\n  - if cpu_count in [0, 39), charge is 0 x cpu_count\n  - if cpu_count in [39, 57), charge is 3 x cpu_count\n  - if cpu_count in [57, 64), charge is 6 x cpu_count',
 "def compute_charge(cpu_type, cpu_count):\n    if cpu_type == 'Intel' and 0 <= cpu_count < 39: return 0 * cpu_count * 1\n    elif cpu_type == 'Intel' and 39 <= cpu_count < 57: return 1 * cpu_count * 1\n    elif cpu_type == 'Intel' and 57 <= cpu_count < 64: return 2 * cpu_count * 1\n    elif cpu_type == 'AMD' and 0 <= cpu_count < 39: 

The following function creates a charge-function for all charges in the given structure.

In [5]:
charges = generate_output_structure(AZURE_CHARGES, AZURE_VARIABLES)

In [6]:
charges

{'cpu_compute': {'description': 'cpu_compute conditions by processor type and CPU load:\n\nfor Intel the following charges apply:\n  - if CPU load in [0.0, 5.0), charge is 0.0 x CPU load\n  - if CPU load in [5.0, 38.3), charge is 1.0 x CPU load\n  - if CPU load in [38.3, 40.5), charge is 2.0 x CPU load\n  - if CPU load in [40.5, 100.0), charge is 3.0 x CPU load\nfor AMD the following charges apply:\n  - if CPU load in [0.0, 5.0), charge is 0.0 x CPU load\n  - if CPU load in [5.0, 38.3), charge is 1.1 x CPU load\n  - if CPU load in [38.3, 40.5), charge is 2.2 x CPU load\n  - if CPU load in [40.5, 100.0), charge is 3.3 x CPU load\nfor ARM the following charges apply:\n  - if CPU load in [0.0, 5.0), charge is 0.0 x CPU load\n  - if CPU load in [5.0, 38.3), charge is 1.2 x CPU load\n  - if CPU load in [38.3, 40.5), charge is 2.4 x CPU load\n  - if CPU load in [40.5, 100.0), charge is 3.6 x CPU load',
  'code': "def compute_charge(cpu_type, cpu_usage):\n    if cpu_type == 'Intel' and 0.0 <=

# Example 2: Generate Document from Structured Charges

In [7]:
from mlisplacement.synthdoc.llm_gen import LLMClient, generate_document_from_json

In [8]:
model = LLMClient(
    model_name="meta-llama/Llama-3.2-1B-Instruct", # Example model, replace with your choice/one for which you have access
)

Device set to use cuda:0


In [12]:
doc, answers, names = generate_document_from_json(
    client=model,
    charges_dict=charges)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [13]:
doc

'# Blue Horizon Airport Charging Policy Document\n\n<table>\n  <tr>\n    <th>Category</th>\n    <th>Service</th>\n    <th>Rate</th>\n    <th>Frequency</th>\n  </tr>\n  <tr>\n    <td>General Admission</td>\n    <td>Upon arrival and departure</td>\n    <td>$X.XX per person</td>\n    <td>Every day</td>\n  </tr>\n  <tr>\n    <td>Security Checkpoint</td>\n    <td>Before boarding</td>\n    <td>$X.XX per person</td>\n    <td>Every day</td>\n  </tr>\n  <tr>\n    <td>Baggage Handling\n\n## <table>\n  <tr>\n    <th>Section</th>\n    <th>Charge\n\n<table border="1" cellpadding="8" cellspacing="0" style="border-collapse: collapse; text-align: center; font-family: Arial, sans-serif; width: 100%;">\n  <caption style="font-weight: bold; font-size: 1.2em; margin-bottom: 10px;">Processor Load Charges by CPU Type</caption>\n  <thead>\n    <tr>\n      <th><strong>CPU Load (%)</th>\n      <th><strong>Processor Type</th>\n      <th><strong>Cargo Rate</th>\n      <th><strong>Private Rate</th>\n      <th><st

In [14]:
answers

["def compute_charge(cpu_type, cpu_usage):\n    if cpu_type == 'Intel' and 0.0 <= cpu_usage < 5.0: return 0 * cpu_usage * 1.0\n    elif cpu_type == 'Intel' and 5.0 <= cpu_usage < 38.3: return 1 * cpu_usage * 1.0\n    elif cpu_type == 'Intel' and 38.3 <= cpu_usage < 40.5: return 2 * cpu_usage * 1.0\n    elif cpu_type == 'Intel' and 40.5 <= cpu_usage < 100.0: return 3 * cpu_usage * 1.0\n    elif cpu_type == 'AMD' and 0.0 <= cpu_usage < 5.0: return 0 * cpu_usage * 1.1\n    elif cpu_type == 'AMD' and 5.0 <= cpu_usage < 38.3: return 1 * cpu_usage * 1.1\n    elif cpu_type == 'AMD' and 38.3 <= cpu_usage < 40.5: return 2 * cpu_usage * 1.1\n    elif cpu_type == 'AMD' and 40.5 <= cpu_usage < 100.0: return 3 * cpu_usage * 1.1\n    elif cpu_type == 'ARM' and 0.0 <= cpu_usage < 5.0: return 0 * cpu_usage * 1.2\n    elif cpu_type == 'ARM' and 5.0 <= cpu_usage < 38.3: return 1 * cpu_usage * 1.2\n    elif cpu_type == 'ARM' and 38.3 <= cpu_usage < 40.5: return 2 * cpu_usage * 1.2\n    elif cpu_type == '

In [15]:
names

['cpu_compute']